In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488493 sha256=3e0c3908f6c539d645f6c6c220320de9a89a1a5719a6be757cb659a4bf81eb16
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [2]:
import pyspark
from pyspark.sql.functions import rand
import datetime

from pyspark.sql.types import StructType, StructField, IntegerType, StringType, BooleanType,FloatType, DoubleType


from pyspark import SparkContext, HiveContext
from pyspark.sql import SparkSession
import time
from pyspark.sql.functions import *

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/11 00:58:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Global Variables

In [2]:
MoneyMule = 50.00

In [3]:
code_not_to_use = [1, 18, 111, 30000, 31813, 32190, 33205, 33206, 37505, 99999 ]

high_risk_categories = ['VH', 'HI', 'PE']

so_scr = ['SO', 'SCR']

## Criteria

In [4]:
A_Criteria = 499.99
B_Criteria = 499.99
C_Criteria = 4999.99
D_Criteria_Lower = 2499.99
D_Criteria_Upper = 4999.99
E_Criteria= 4999.99
F_Criteria = 499.99
G_Criteria = 2499.99

Y_Criteria = 500.00
Z_Criteria = 5999.99

Minor = 16
MuleLower = 17
MuleUpper = 25

## Read the files

In [6]:
member_path='/Users/Marium_Faheem/Library/CloudStorage/OneDrive-McKinsey&Company/Desktop/learning/maham-df/ScionMemberList.csv'
trans_path='/Users/Marium_Faheem/Library/CloudStorage/OneDrive-McKinsey&Company/Desktop/learning/maham-df/ScionTransactionList.csv'

In [8]:
member_df = spark.read.option("header", True).csv(member_path)

trans_df = spark.read.option("header", True).csv(trans_path)

# Read Scion Transaction File

In [9]:
trans_df.show()

+--------------+---------------+---------+--------------+----------------+-------------------+--------------------+----------------+----------------+
|Account Number|Financial Month|     Date|Transaction No|Transaction Type|Trans Type Category|    Transaction Note|Trans In Amount |Trans Out Amount|
+--------------+---------------+---------+--------------+----------------+-------------------+--------------------+----------------+----------------+
|             7|         23-Dec|01-Dec-23|      46980801|              SD|                EFT|       Not Available|             400|               0|
|             7|         23-Dec|08-Dec-23|      46997807|              SD|                EFT|       Not Available|             400|               0|
|             7|         23-Dec|15-Dec-23|      47013444|              SD|                EFT|       Not Available|             400|               0|
|             7|         23-Dec|22-Dec-23|      47030043|              SD|                EFT|      

In [10]:
trans_df.printSchema()

root
 |-- Account Number: string (nullable = true)
 |-- Financial Month: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Transaction No: string (nullable = true)
 |-- Transaction Type: string (nullable = true)
 |-- Trans Type Category: string (nullable = true)
 |-- Transaction Note: string (nullable = true)
 |-- Trans In Amount : string (nullable = true)
 |-- Trans Out Amount: string (nullable = true)



## Data Pre-Processing

In [14]:
trans_df = trans_df.withColumnRenamed('Trans In Amount ', 'TransInAmount')
trans_df = trans_df.withColumnRenamed('Trans Out Amount', 'TransOutAmount')


In [15]:
trans_df.show()

+--------------+---------------+---------+--------------+----------------+-------------------+--------------------+-------------+--------------+
|Account Number|Financial Month|     Date|Transaction No|Transaction Type|Trans Type Category|    Transaction Note|TransInAmount|TransOutAmount|
+--------------+---------------+---------+--------------+----------------+-------------------+--------------------+-------------+--------------+
|             7|         23-Dec|01-Dec-23|      46980801|              SD|                EFT|       Not Available|          400|             0|
|             7|         23-Dec|08-Dec-23|      46997807|              SD|                EFT|       Not Available|          400|             0|
|             7|         23-Dec|15-Dec-23|      47013444|              SD|                EFT|       Not Available|          400|             0|
|             7|         23-Dec|22-Dec-23|      47030043|              SD|                EFT|       Not Available|          400| 

In [17]:
from pyspark.sql.functions import *
pattern = r"\,"
trans_df = trans_df.withColumn(
    "TransInAmount",
    regexp_replace(trans_df["TransInAmount"], pattern, "")  # Replace comma with empty string
)

trans_df = trans_df.withColumn(
    "TransOutAmount",
    regexp_replace(trans_df["TransOutAmount"], pattern, "")  # Replace comma with empty string
)

In [18]:
trans_df.show()

+--------------+---------------+---------+--------------+----------------+-------------------+--------------------+-------------+--------------+
|Account Number|Financial Month|     Date|Transaction No|Transaction Type|Trans Type Category|    Transaction Note|TransInAmount|TransOutAmount|
+--------------+---------------+---------+--------------+----------------+-------------------+--------------------+-------------+--------------+
|             7|         23-Dec|01-Dec-23|      46980801|              SD|                EFT|       Not Available|          400|             0|
|             7|         23-Dec|08-Dec-23|      46997807|              SD|                EFT|       Not Available|          400|             0|
|             7|         23-Dec|15-Dec-23|      47013444|              SD|                EFT|       Not Available|          400|             0|
|             7|         23-Dec|22-Dec-23|      47030043|              SD|                EFT|       Not Available|          400| 

# Calculate Columns from `Transaction File`

In [32]:
trans_df = trans_df.withColumn("TransactionTotal", col("TransInAmount") - col("TransOutAmount"))

trans_df = trans_df.withColumn(
    "TransactionPositiveTotal",
    when(col("TransactionTotal").isNull(), "").otherwise(abs(col("TransactionTotal")))
)

trans_df = trans_df.withColumn(
    "TransForMoneyMuleAndCombined",
    when(col("TransactionPositiveTotal") > MoneyMule, col("TransactionPositiveTotal")).otherwise(0)
)

trans_df = trans_df.withColumn(
    "Trans Incl",
    when(col("TransactionPositiveTotal") > 0, "Yes").otherwise("No")
)

trans_df = trans_df.withColumn(
    "Account_Not_In_Use",
    when(col("Account Number").isin(code_not_to_use), "No").otherwise("Yes")
)

trans_df = trans_df.withColumn(
    "SO_SCR",
    when(col("Transaction Type").isin(so_scr), col('Transaction Type')).otherwise("")
)

# previous value = SCR and current value = s0 and next value = s0

# trans_df = trans_df.withColumn(
#     "No1",
#     when(col("SO_SCR").isin(so_scr), "Y").otherwise("")
# )

# trans_df = trans_df.withColumn("index", monotonically_increasing_id())

In [33]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import col, lag, lead

windowSpec = Window.orderBy("Account Number")

# Add columns for previous and next values
df_with_prev_next = trans_df.withColumn("prev_value", lag("SO_SCR", 1).over(windowSpec)) \
    .withColumn("next_value", lead("SO_SCR", 1).over(windowSpec))

# Filter based on your conditions
df_filtered = df_with_prev_next.filter(
    (col("prev_value") == "SCR") &
    (col("SO_SCR") == "s0") &
    (col("next_value") == "s0")
)

# Show the result
df_with_prev_next.show(truncate=False)
df_filtered.write.csv('/Users/Marium_Faheem/Library/CloudStorage/OneDrive-McKinsey&Company/Desktop/learning/maham-df/df_filtered.csv', header=True, mode="overwrite")

24/03/11 01:08:43 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/11 01:08:43 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/11 01:08:43 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/11 01:08:43 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/11 01:08:43 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/11 01:08:43 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/11 0

+--------------+---------------+---------+--------------+----------------+-------------------+----------------+-------------+--------------+----------------+------------------------+----------------------------+----------+------------------+------+---+-----+----------+----------+
|Account Number|Financial Month|Date     |Transaction No|Transaction Type|Trans Type Category|Transaction Note|TransInAmount|TransOutAmount|TransactionTotal|TransactionPositiveTotal|TransForMoneyMuleAndCombined|Trans Incl|Account_Not_In_Use|SO_SCR|No1|index|prev_value|next_value|
+--------------+---------------+---------+--------------+----------------+-------------------+----------------+-------------+--------------+----------------+------------------------+----------------------------+----------+------------------+------+---+-----+----------+----------+
|NULL          |NULL           |NULL     |NULL          |NULL            |NULL               |NULL            |NULL         |NULL          |NULL            |

24/03/11 01:08:44 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/11 01:08:44 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [17]:
trans_df.show()

+--------------+---------------+---------+--------------+----------------+-------------------+--------------------+-------------+--------------+----------------+------------------------+----------------------------+----------+------------------+------+---+-----+
|Account Number|Financial Month|     Date|Transaction No|Transaction Type|Trans Type Category|    Transaction Note|TransInAmount|TransOutAmount|TransactionTotal|TransactionPositiveTotal|TransForMoneyMuleAndCombined|Trans Incl|Account_Not_In_Use|SO_SCR|No1|index|
+--------------+---------------+---------+--------------+----------------+-------------------+--------------------+-------------+--------------+----------------+------------------------+----------------------------+----------+------------------+------+---+-----+
|             7|         23-Dec|01-Dec-23|      46980801|              SD|                EFT|       Not Available|          400|             0|           400.0|                   400.0|                       40

In [18]:
# trans_df.withColumn(
#     "No2",
#     when((col("No1") == "Y") & (col('SO_SCR') == "SO" || "SCR"), "Y").otherwise("")
# ).show()
# trans_df.withColumn(
#     "No2",
#     when((col("No1") == "Y") & (col('SO_SCR').isin("SO", "SCR")), "Y").otherwise("")
# ).show()



trans_df.withColumn(
    "No2",
    when((col("No1") == "Y") & ((col('SO_SCR') == "SO") | (col('SO_SCR') == "SCR")), "Y").otherwise("")
).show()

+--------------+---------------+---------+--------------+----------------+-------------------+--------------------+-------------+--------------+----------------+------------------------+----------------------------+----------+------------------+------+---+-----+---+
|Account Number|Financial Month|     Date|Transaction No|Transaction Type|Trans Type Category|    Transaction Note|TransInAmount|TransOutAmount|TransactionTotal|TransactionPositiveTotal|TransForMoneyMuleAndCombined|Trans Incl|Account_Not_In_Use|SO_SCR|No1|index|No2|
+--------------+---------------+---------+--------------+----------------+-------------------+--------------------+-------------+--------------+----------------+------------------------+----------------------------+----------+------------------+------+---+-----+---+
|             7|         23-Dec|01-Dec-23|      46980801|              SD|                EFT|       Not Available|          400|             0|           400.0|                   400.0|             

In [19]:
# Define a function to check the value of "No1" for the previous record
def check_previous_no1(index, no1_values):
    if index == 0:
        return ""
    else:
        return no1_values[index - 1]

# Register the function as a UDF
spark.udf.register("check_previous_no1", check_previous_no1)

# Add a new column "prev_No1" containing the value of "No1" from the previous record
trans_df = trans_df.withColumn("prev_No1", expr("check_previous_no1(index, collect_list(No1) over (order by index))"))

# Apply the condition based on the previous records of "No1" and "SO_SCR"
trans_df = trans_df.withColumn(
    "No2",
    when((col("No1") == "Y") & ((col('SO_SCR') == "SO") | (col('SO_SCR') == "SCR")) & (col("prev_No1") == "Y"), "Y").otherwise("")
)

# Drop the intermediate column "prev_No1" if not needed
trans_df = trans_df.drop("prev_No1")

# Show the DataFrame with the new "No2" column
trans_df.show()

+--------------+---------------+---------+--------------+----------------+-------------------+--------------------+-------------+--------------+----------------+------------------------+----------------------------+----------+------------------+------+---+-----+---+
|Account Number|Financial Month|     Date|Transaction No|Transaction Type|Trans Type Category|    Transaction Note|TransInAmount|TransOutAmount|TransactionTotal|TransactionPositiveTotal|TransForMoneyMuleAndCombined|Trans Incl|Account_Not_In_Use|SO_SCR|No1|index|No2|
+--------------+---------------+---------+--------------+----------------+-------------------+--------------------+-------------+--------------+----------------+------------------------+----------------------------+----------+------------------+------+---+-----+---+
|             7|         23-Dec|01-Dec-23|      46980801|              SD|                EFT|       Not Available|          400|             0|           400.0|                   400.0|             

In [20]:
trans_df.withColumn(
    "No3",
    when((col("No1") == "Y") & ((col('No2') == "Y")), "Y").otherwise("")
)

DataFrame[Account Number: string, Financial Month: string, Date: string, Transaction No: string, Transaction Type: string, Trans Type Category: string, Transaction Note: string, TransInAmount: string, TransOutAmount: string, TransactionTotal: double, TransactionPositiveTotal: string, TransForMoneyMuleAndCombined: string, Trans Incl: string, Account_Not_In_Use: string, SO_SCR: string, No1: string, index: bigint, No2: string, No3: string]

In [21]:
from pyspark.sql.functions import lag, col, when
from pyspark.sql.window import Window

# Define a window specification to order the records
windowSpec = Window.orderBy('index')

# Add a new column "prev_Transaction_Total" containing the value of "Transaction Total" from the previous record
trans_df = trans_df.withColumn("prev_Transaction_Total", lag(col("TransactionTotal"), 1).over(windowSpec))

# Add a new column "No3" based on conditions of "No1" and "No2"
trans_df = trans_df.withColumn(
    "No3",
    when((col("No1") == "Y") & (col("No2") == "Y") & (col("TransactionTotal") == col("prev_Transaction_Total")), "Y").otherwise("")
)

# Drop the intermediate column "prev_Transaction_Total" if not needed
trans_df = trans_df.drop("prev_Transaction_Total")

# Show the DataFrame with the new "No3" column



In [22]:
trans_df = trans_df.drop("prev_Transaction_Total")

In [23]:
trans_df.show()

+--------------+---------------+---------+--------------+----------------+-------------------+--------------------+-------------+--------------+----------------+------------------------+----------------------------+----------+------------------+------+---+-----+---+---+
|Account Number|Financial Month|     Date|Transaction No|Transaction Type|Trans Type Category|    Transaction Note|TransInAmount|TransOutAmount|TransactionTotal|TransactionPositiveTotal|TransForMoneyMuleAndCombined|Trans Incl|Account_Not_In_Use|SO_SCR|No1|index|No2|No3|
+--------------+---------------+---------+--------------+----------------+-------------------+--------------------+-------------+--------------+----------------+------------------------+----------------------------+----------+------------------+------+---+-----+---+---+
|             7|         23-Dec|01-Dec-23|      46980801|              SD|                EFT|       Not Available|          400|             0|           400.0|                   400.0| 

# Member File

In [24]:
member_df.show()

+--------------+------------+------------------+--------------------+---+-----------+--------------+---------+-----------+
|Account Number|Account Type| Employment Status| Employment Position|Age|Nationality|Account Status|Risk Code|Date Joined|
+--------------+------------+------------------+--------------------+---+-----------+--------------+---------+-----------+
|             7|      SINGLE|EMPLOYED FULL-TIME| CONSTRUCTION WORKER| 40|      IRISH|        ACTIVE|       LO|  11-Jan-90|
|            10|      SINGLE|           RETIRED|       Not Available| 84|      IRISH|        ACTIVE|       LO|  24-Jan-90|
|            14|      SINGLE|           RETIRED|STATE PENSION NON...| 78|      IRISH|        ACTIVE|       LO|  16-Feb-79|
|            20|      SINGLE|           RETIRED|       STATE PENSION| 78|      IRISH|        ACTIVE|       LO|  30-Oct-67|
|            33|      SINGLE|           RETIRED|       Not Available| 75|      IRISH|        ACTIVE|       LO|  18-Dec-67|
|            42|

# Joining the table

In [25]:
member_trans_df = trans_df.join(member_df,trans_df['Account Number']==member_df['Account Number'], how="inner")


In [26]:
member_trans_df.show()

+--------------+---------------+---------+--------------+----------------+-------------------+--------------------+-------------+--------------+----------------+------------------------+----------------------------+----------+------------------+------+---+-----+---+---+--------------+------------+------------------+--------------------+---+-----------+--------------+---------+-----------+
|Account Number|Financial Month|     Date|Transaction No|Transaction Type|Trans Type Category|    Transaction Note|TransInAmount|TransOutAmount|TransactionTotal|TransactionPositiveTotal|TransForMoneyMuleAndCombined|Trans Incl|Account_Not_In_Use|SO_SCR|No1|index|No2|No3|Account Number|Account Type| Employment Status| Employment Position|Age|Nationality|Account Status|Risk Code|Date Joined|
+--------------+---------------+---------+--------------+----------------+-------------------+--------------------+-------------+--------------+----------------+------------------------+----------------------------+-

In [27]:
member_trans_df.filter(member_trans_df['Risk Code'] == "HI").show()

+--------------+---------------+---------+--------------+----------------+-------------------+----------------+-------------+--------------+----------------+------------------------+----------------------------+----------+------------------+------+---+-----+---+---+--------------+------------+-----------------+--------------------+---+-----------+--------------+---------+-----------+
|Account Number|Financial Month|     Date|Transaction No|Transaction Type|Trans Type Category|Transaction Note|TransInAmount|TransOutAmount|TransactionTotal|TransactionPositiveTotal|TransForMoneyMuleAndCombined|Trans Incl|Account_Not_In_Use|SO_SCR|No1|index|No2|No3|Account Number|Account Type|Employment Status| Employment Position|Age|Nationality|Account Status|Risk Code|Date Joined|
+--------------+---------------+---------+--------------+----------------+-------------------+----------------+-------------+--------------+----------------+------------------------+----------------------------+----------+----

# Calculate Columns from `Member_trans_df`

In [28]:
member_trans_df = member_trans_df.withColumn(
    "HighRisk",
    when(col("Risk Code").isin(high_risk_categories), "Yes").otherwise("No")
)

# Account_Not_In_Use ki jaGw code_in_use ayeGw
member_trans_df = member_trans_df.withColumn("GreaterThan",
                               when((col("Account_Not_In_Use")=='Yes') & (col("Trans Incl")=='Yes') & ((col("TransInAmount") >= D_Criteria_Lower) & (col("TransInAmount")<= D_Criteria_Upper)), "Yes")
                               .otherwise("No"))

# Display the `member_trans_df` DataFrame

In [ ]:
member_trans_df.show()

### Verification of the Column Created

In [ ]:
member_trans_df.filter(col('GreaterThan') == "Yes").show(5)

In [ ]:
member_trans_df.filter(col('HighRisk') == "Yes").show(5)

In [ ]:
# code no tot use = yes
# trans incl yes
# high risk yes
# trans postive total > A_Crtiera


# Creteria Creation

In [ ]:
member_trans_df = member_trans_df.withColumn("A",
                                               when((col("Account_Not_In_Use")=="Yes") # Account_Not_In_Use ki jaga code_in_use ayeGw column
                                                    & (col("HighRisk")=="Yes")
                                                    & (col("TransactionPositiveTotal") > A_Criteria ), "A")
                                               .otherwise(None))

In [ ]:
counts_of_A = member_trans_df.filter(member_trans_df['A'] == 'A').count()
print("Count of records where column 'A' equals 'A':", counts_of_A)
print("==========================================================")